In [347]:
import opstrat as op
import pandas as pd
import numpy as np
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
import seaborn as sns

import json
with open('configcal.json') as config_file:
     data = json.load(config_file)
print(data)

last_price          = data['Current_Price']
annual_drift        = data['Annual_Drift']
annual_volatility   = data['Annual_Volatiltiy']
time_period         = data['Time_Period']
simu                = data['No_of_days']
r                   = data['rate_return']
sigma               = data['sigma']
price               = data['Current_Price']

df = pd.DataFrame()

last_price_list = [last_price]
diff_list       = []
tenor_list_1st     = []
tenor_list_2nd     = []
count           = 0
count_list      = []
while count < simu:
    count = count + 1
    difference = last_price*annual_drift*time_period + last_price*annual_volatility*(time_period**(0.5))*norm.ppf(random.random())

    last_price = last_price + difference
    tenor1st      = ((4-count)/(7*52))
    tenor2nd      = ((11-count)/(7*52))
    last_price_list.append(last_price)
    diff_list.append(difference)
    count_list.append(count)

    tenor_list_1st.append(tenor1st)
    tenor_list_2nd.append(tenor2nd)

last_price_list.pop(-1)
#tenor_list_1st.pop(-1)
#tenor_list_2nd.pop(-1)
df['Count'] = count_list
df['Price'] = last_price_list
df['Change_price'] = diff_list
df['tenor_1st_expiry'] = tenor_list_1st
df['tenor_2nd_expiry'] = tenor_list_2nd



{'Current_Price': 17344, 'Annual_Drift': 0.07, 'Annual_Volatiltiy': 0.15, 'Time_Period': 0.003968254, 'rate_return': 0.07, 'sigma': 0.117, 'No_of_days': 3}


In [348]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import math



#list1 = [int(round(price, -2))+200,int(round(price, -2))+200]

datalist = dict()

k = int(round(price, -2))+400

call_option     = []
call_thetalist  = []
call_gammalist  = []
call_deltalist  = []
call_rholist    = []
call_vegalist   = [] 
for last_price,T in zip(df['Price'],df['tenor_1st_expiry']):
    d1         = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
    d2         = d1-sigma*(T**0.5)
    N_d1       = norm.cdf(d1)
    N_d2       = norm.cdf(d2)
    N_d1_      = 1-norm.cdf(d1)
    N_d2_      = 1-norm.cdf(d2)
    call       = last_price*N_d1-k*math.exp(-r*T)*N_d2
    #put        = k*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
    call_delta = -norm.cdf(d1)
    call_gamma = -norm.pdf(d1)/(last_price*sigma*sqrt(T))
    call_vega  = -0.01*(last_price*norm.pdf(d1)*sqrt(T))
    call_theta = (-0.01*((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2)))/simu
    call_rho   = -0.01*(k*T*exp(-r*T)*norm.cdf(d2))
    
    call_option.append(call)
    call_thetalist.append(call_theta)
    call_gammalist.append(call_gamma)
    call_deltalist.append(call_delta)
    call_rholist.append(call_rho)
    call_vegalist.append(call_vega)

    datalist['CE_Sell']      = call_option
    datalist['theta_Sell']   = call_thetalist
    datalist['gamma_Sell']   = call_gammalist
    datalist['delta_Sell']   = call_deltalist
    datalist['rho_Sell']     = call_rholist
    datalist['vega_Sell']    = call_vegalist


k = int(round(price, -2))+400

call_option     = []
call_thetalist  = []
call_gammalist  = []
call_deltalist  = []
call_rholist    = []
call_vegalist   = [] 
for last_price,T in zip(df['Price'],df['tenor_2nd_expiry']):
    d1         = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
    d2         = d1-sigma*(T**0.5)
    N_d1       = norm.cdf(d1)
    N_d2       = norm.cdf(d2)
    N_d1_      = 1-norm.cdf(d1)
    N_d2_      = 1-norm.cdf(d2)
    call       = -(last_price*N_d1-k*math.exp(-r*T)*N_d2)
    #put        = k*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
    call_delta = norm.cdf(d1)
    call_gamma = norm.pdf(d1)/(last_price*sigma*sqrt(T))
    call_vega  = 0.01*(last_price*norm.pdf(d1)*sqrt(T))
    call_theta = (0.01*((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2)))/simu
    call_rho   = 0.01*(k*T*exp(-r*T)*norm.cdf(d2))
    
    call_option.append(call)
    call_thetalist.append(call_theta)
    call_gammalist.append(call_gamma)
    call_deltalist.append(call_delta)
    call_rholist.append(call_rho)
    call_vegalist.append(call_vega)

    datalist['CE_Buy']      = call_option
    datalist['theta_Buy']   = call_thetalist
    datalist['gamma_Buy']   = call_gammalist
    datalist['delta_Buy']   = call_deltalist
    datalist['rho_Buy']     = call_rholist
    datalist['vega_Buy']    = call_vegalist



df1 = pd.DataFrame(datalist)

In [349]:
df = pd.concat([df,df1],axis=1)

In [350]:
#Calculating the total premium paid or recieved 
filter_col = [col for col in list(df) if col.startswith(('PE_','CE_'))]
df['Total_Prem'] = 0 # initialize value, otherwise throws KeyErrorx
for d in df[filter_col]:
    df['Total_Prem'] = df[filter_col].sum(axis=1)   # Total_PP = total put premium 


#Calculating the total Profit and Loss

df['Total_P&L'] = df['Total_Prem'][0] - df['Total_Prem']


#Calculating greeks for overall data

list1 = ['theta','gamma','delta','rho','vega']

datalist4 = dict()

for data in list1:
    filter_col = [col for col in list(df) if col.startswith(data)]
    df['Port_'+ data] = 0 # initialize value, otherwise throws KeyErrorx
    for d in df[filter_col]:
        df['Port_'+ data] = df[filter_col].sum(axis=1)    

#final_df = pd.concat([df,df],axis = 1)

In [351]:
df.to_csv('Calender.csv')

In [352]:
df

,Count,Price,Change_price,tenor_1st_expiry,tenor_2nd_expiry,CE_Sell,theta_Sell,gamma_Sell,delta_Sell,rho_Sell,...,delta_Buy,rho_Buy,vega_Buy,Total_Prem,Total_P&L,Port_theta,Port_gamma,Port_delta,Port_rho,Port_vega
0,1,17344.000000,241.205880,0.008242,0.027473,2.288842,2.797293,-0.000389,-0.031924,-0.045445,...,0.173901,0.820050,7.380783,-28.872970,0.000000,-3.138082,0.000375,0.141978,0.774605,6.252873
1,2,17585.205880,-118.718372,0.005495,0.024725,21.639416,15.415161,-0.002045,-0.241474,-0.232128,...,0.401140,1.721500,10.690873,-69.976138,41.103168,5.359039,-0.000850,0.159666,1.489373,6.625111
2,3,17466.487508,174.985631,0.002747,0.021978,0.634112,2.742838,-0.000384,-0.016537,-0.007918,...,0.251973,0.957324,8.262824,-44.630446,15.757476,-5.604712,0.000669,0.235437,0.949406,7.885874


In [353]:
int(round(price, -2))+100

17400

In [354]:
15.189037/13.262674

1.1452469539702175

In [355]:
31.355457/28.220004

1.1111074612179361

In [356]:
54.288697/39.376793

1.3786977776478648

In [357]:
int(round(price, -2))+400

17700